# Classificação Logística

Propomos utilizar **Polaridade-Subjetividade**, junto a **Presença-Ausência** de um conjunto de temas em cada Tweet como parte dos preditores; junto a isso, a **Contagem de Retweets**, **Contagem de Favoritos**, **Presença de Mídias**, por exemplo.

Propomos ainda usar um fator de retardo $k$ para caracterizar a influencia retardada das postagens. Caso o ajuste significativo ocorra sob um fator $k \neq 0$, isto é, caso alterações relevantes sejam observadas após as postagens, e com coeficientes significativos, podemos discutir a influencia dos tweets como possível; caso o retardo seja nulo, ou pequeno, podemos discutir a existência de correlação sem causalidade.

**Referencias Utilizadas**:
  * **Constantin Colonescu**. The Effects of Donald Trump’s Tweets on US Financial and Foreign Exchange Markets. Athens Journal of Business and Economics. Disponível em: www.athensjournals.gr/business/2018-1-X-Y-Colonescu.pdf

In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import seaborn as sns

import matplotlib
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegressionCV
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from scipy.stats import norm

from datetime import datetime
from datetime import timedelta

%matplotlib inline

## Preparação dos dados

Inclui as etapas de standandização, separação, e agregação segundo diferentes métodos. Primeiramente corrigimos os problemas de tipos de dados, depois transformamos as nossas variáveis qualitativas em **dummies**. Com isso agrupamos por data pegando a média de todas as colunas como função de agrupamento. Para essa primeira análise estaremos analisando os tweets de Jair Bolsonaro com relação ao câmbio do dólar.

Primeiro passo é ler nossos dados.

In [2]:
# ler os dados
tweets_data = pd.read_pickle("..//data//tweets//final_tweets_data.pkl")
tweets_data = tweets_data.drop(["full_text", "full_text_en"], axis=1)
economic_data = pd.read_csv("..//data//economic_data//economic_time_series.csv", sep=";", index_col=0)

Usamos um formato padrão de datas e ajustamos as séries temporais.

In [3]:
# usar o formato de data padrão "datetime"
economic_data.index = pd.to_datetime(economic_data.index)

tweets_data["date"]  = tweets_data.year.apply(str) + "-"
tweets_data["date"] += tweets_data.month.apply(str) + "-"
tweets_data["date"] += tweets_data.day.apply(str)
tweets_data["date"]  = pd.to_datetime(tweets_data["date"])

Adicionamos algumas *dummy-variables* convertendo um conjunto de variáveis qualitativas dos preditores em colunas numéricas. Também manteremos essa análise apenas para o twitter de jairbolsonaro.

In [4]:
# filtrar nossos dados para o que queremos usar
tweets_data_cla = tweets_data.iloc[tweets_data.name.values=='jairbolsonaro',:]
tweets_data_cla = tweets_data_cla[['date','retweet_count','polarity','subjectivity','favorite_count','topic','media_type']]
tweets_data_cla['media_type'] = tweets_data_cla['media_type'].fillna('none')
# transformar as duas colunas em dummy
cols_to_dumm = ["topic","media_type"]
for col in cols_to_dumm:
    dummies = pd.get_dummies(tweets_data_cla[col], prefix=col, drop_first=True)
    tweets_data_cla = pd.concat([tweets_data_cla.drop(col, axis=1), dummies], axis=1)

Para primeira análise, agregamos alguns dados com relação à data, de forma a pegar a média de todos os dados quantitativos naquela data e a soma de todos os dados qualitativos (ou dummy).

In [5]:
# agregar por data usando a média nas seguintes colunas
dayly_agg_columns = ["date",'retweet_count','polarity','subjectivity','favorite_count']
dayly_agregated = tweets_data_cla[dayly_agg_columns].groupby("date", as_index=False).mean()

dayly_agregated.columns = [col + "_mean" for col in dayly_agg_columns]
dayly_agregated = dayly_agregated.set_index("date_mean")

# agregar por data usando a soma nas seguintes colunas
# as variáveis dummy são agregadas por soma para depois serem normalizadas de modo a somar um em cada categoria
dayly_agg_columns2 = ['date','media_type_photo','media_type_video']+['topic'+f'_{i}' for i in range(1,10)]
dayly_agregated2 = tweets_data_cla[dayly_agg_columns2].groupby("date", as_index=False).sum()

dayly_agregated2.columns = dayly_agg_columns2
dayly_agregated2 = dayly_agregated2.set_index("date")

# colocar os dados juntos
tweets_data_cla = pd.concat([dayly_agregated, dayly_agregated2], axis=1)

Redimensionalizando nosso dados para média $0$ e desvio padrão $1$ e também tornando os tópicos e midías como porcentagem da soma total.

In [6]:
# transformando os dados quantitativos
cols_to_scale = ['retweet_count_mean','polarity_mean','subjectivity_mean','favorite_count_mean']
for col in cols_to_scale:
    tweets_data_cla[col] = StandardScaler().fit_transform(tweets_data_cla[col].values.reshape(-1,1))

# transformando as dummies de modo a somar 1
for row in range(0,len(tweets_data_cla)):
    #media
    if tweets_data_cla.iloc[row,4:6].sum() != 0:
        tweets_data_cla.iloc[row,4:6] = tweets_data_cla.iloc[row,4:6]/tweets_data_cla.iloc[row,4:6].sum()
    #topic
    if tweets_data_cla.iloc[row,6:15].sum() != 0:
        tweets_data_cla.iloc[row,6:15] = tweets_data_cla.iloc[row,6:15]/tweets_data_cla.iloc[row,6:15].sum()

Lidando com o problema da compreensão das datas dos tweets contra os dos dados econômicos.

In [7]:
# escolha os dados econômicos no mesmo alcance que os tweets
minD = tweets_data_cla.index.min()
maxD = tweets_data_cla.index.max()

dt_day = timedelta(days=1)
economic_data = economic_data[economic_data.index >= minD - dt_day]
economic_data = economic_data[economic_data.index <= maxD]

Primeiramente, vamos pegar os index dos dados econômicos do câmbio do dólar, isto é, todas posições em que ele não é NaN.

In [8]:
# pegue as datas em que temos o valor do dolar naquele dia e seus correspondentes index nos dados dos tweets
economic_index = 'dollar'
dates = economic_data.index[~np.isnan(economic_data[economic_index])]
dates_index_tweets = [tweets_data_cla.index[row] in dates for row in range(0,len(tweets_data_cla))]

Agora, $k$ é o fator de quantas observações anterior impactam na atual (prever $X_{n+1}$ usando $X_{n-k},...,X_{n}$). Caso a nossa observação não possua $k$ observações anteriores, são usadas as que tem.

In [9]:
# passe pelos dados agregando os k dados anteriores
# transformamos os dados em np.array e depois de volta a dataframe devido à velocidade das operações
kMax = 2
model_data = tweets_data_cla.iloc[dates_index_tweets,:]
model_data_values = model_data.values
for date_ind in range(0,len(model_data.index)):
    row_values = model_data_values[date_ind,:]
    values_found = 1
    for k in range(1,kMax+1):
        cur_date = model_data.index[date_ind] - timedelta(days=k)
        if cur_date in tweets_data_cla.index:
            row_values = row_values + tweets_data_cla.values[list(tweets_data_cla.index).index(cur_date),:]
            values_found += 1
    model_data_values[date_ind,:] = row_values/values_found
model_data = pd.DataFrame(model_data_values,columns=model_data.columns, index = model_data.index)
# adicione os dados econômicos
model_data[economic_index] = economic_data[economic_index][~np.isnan(economic_data[economic_index])]

Para passar para o modelo nos falta apenas modificar os dados que queremos prever: ao invés de prever absolutamente o crescimento do índice, tentaremos modelar se houve crescimento ou não. 

In [10]:
# transformar o crescimento do dólar em 1 caso aumentou 0 caso contrário
eco_data = model_data[economic_index].values

for row in range(len(eco_data)-1,0,-1):
    eco_data[row] = int(eco_data[row]>=eco_data[row-1])

model_data[economic_index] = eco_data
model_data = model_data.drop([model_data.index[0]])

## Modelagem


Separar os dados em treino e teste de modo que em cada um tenha a mesma quantidade de $1$'s e $0$'s.

In [11]:
data_train, data_test = train_test_split(model_data, test_size = 0.2, stratify=model_data[economic_index],random_state=42)

Separar $X$ de $y$.

In [12]:
x_train = data_train.drop(columns=economic_index)
x_test = data_test.drop(columns=economic_index)
y_train = np.array(data_train[economic_index])
y_test = np.array(data_test[economic_index])

O primeiro modelo a ser criado é uma Regressão Logística usando Cross-Validation.

In [13]:
x_train_t = x_train.as_matrix()
y_train_t = np.array(y_train)
x_test_t = x_test.as_matrix()
y_test_t = np.array(y_test)

lr = LogisticRegressionCV(solver='liblinear', multi_class='ovr',
                            penalty='l2', max_iter=100000, cv=10).fit(x_train_t,y_train_t)

lr_train_accuracy = lr.score(x_train_t, y_train_t)
lr_test_accuracy = lr.score(x_test_t, y_test_t)

scores = {'train': lr_train_accuracy, 
          'test': lr_test_accuracy}

print(scores)

{'train': 0.5643153526970954, 'test': 0.5573770491803278}


Podemos perceber que o modelo teve uma boa acurácia nos dados de teste. Um outro modelo é a mesma Regressão Logística usando Cross-Validation só que, desta vez, adicionaremos termos polinomiais. No caso são adicionados os termos quadráticos dos preditores quantitativos e os termos de interação entre cada preditor quantitativo com cada preditor qualitativo.

In [14]:
# columns_to_poly são os dados quantitativos e columns_to_interact são os qualitativos
columns_to_poly = ['retweet_count_mean', 'polarity_mean', 'subjectivity_mean','favorite_count_mean']
columns_to_interact = ['media_type_photo', 'media_type_video','topic_1', 'topic_2', 'topic_3',
                       'topic_4', 'topic_5', 'topic_6','topic_7', 'topic_8', 'topic_9']

# nossos transformadores com termos de interação sim e não
poly_no_int = PolynomialFeatures(2, include_bias=False,interaction_only=False)
poly_int = PolynomialFeatures(2, include_bias=False,interaction_only=True)

# transformar os dados X do treino e teste
dataframe_to_concat_train = []
dataframe_to_concat_test = []
for col in columns_to_poly:
    dataframe_to_concat_train.append(pd.DataFrame(poly_no_int.fit_transform(x_train[col].values.reshape(-1,1)),
                                           columns = [col,col+'**2']))
    dataframe_to_concat_test.append(pd.DataFrame(poly_no_int.fit_transform(x_test[col].values.reshape(-1,1)),
                                           columns = [col,col+'**2']))

for col_poly in columns_to_poly:
    for col_int in columns_to_interact:
        k = poly_int.fit_transform(x_train[[col_poly,col_int]])
        dataframe_to_concat_train.append(pd.DataFrame(poly_int.fit_transform(x_train[[col_poly,col_int]])[:,2],
                                           columns = ['int_'+col_poly+'/'+col_int]))
        dataframe_to_concat_test.append(pd.DataFrame(poly_int.fit_transform(x_test[[col_poly,col_int]])[:,2],
                                           columns = ['int_'+col_poly+'/'+col_int]))        

# adiciona as dummies
dataframe_to_concat_train.append(x_train[columns_to_interact].reset_index(drop=True))
dataframe_to_concat_test.append(x_test[columns_to_interact].reset_index(drop=True))

# concatena todas nossas colunas
x_train_poly = pd.concat(dataframe_to_concat_train, axis=1)
x_test_poly = pd.concat(dataframe_to_concat_test, axis=1)

Com os dados modificados, podemos agora fitar o modelo.

In [15]:
x_train_t_poly = x_train_poly.as_matrix()
x_test_t_poly = x_test_poly.as_matrix()

lrPoly = LogisticRegressionCV(solver='liblinear', multi_class='ovr',
                            penalty='l2', max_iter=100000, cv=10).fit(x_train_t_poly,y_train_t)

lr_train_accuracy = lrPoly.score(x_train_t_poly, y_train_t)
lr_test_accuracy = lrPoly.score(x_test_t_poly, y_test_t)

scoresPoly = {'train': lr_train_accuracy, 
          'test': lr_test_accuracy}

print(scoresPoly)

{'train': 0.5892116182572614, 'test': 0.5901639344262295}


Podemos ver que o modelo teve uma performance melhor no treino do que o anterior, porém pior no teste. Isso provavelmente acontece devido ao overfitting que preditores polinomiais trazem.

Duas outras medidas interessantes para se analisar nesse caso são:

1) A porcentagem de valores $1$ (crescimento do índice) com relação ao total;
2) A acurácia do modelo na classe $0$ (geralmente é a não dominante);

In [16]:
lr_class_accuracy = lrPoly.score(np.concatenate([x_train_t_poly[y_train_t==0,:],x_test_t_poly[y_test_t==0,:]]),
                                                      np.concatenate([y_train_t[y_train_t==0],y_test_t[y_test_t==0]]))
value_percentage = model_data[economic_index].sum()/len(model_data[economic_index])
print("Acurácia na classe 0:",lr_class_accuracy,"\nPorcentagem de 1s",value_percentage)

Acurácia na classe 0: 0.2986111111111111 
Porcentagem de 1s 0.5231788079470199


Podemos ver que nosso modelo possue baixa acurácia na classe $0$ (índice decresceu). Também é possível analisar que a acurácia de um modelo que só "chutasse" 1's seria quase a mesma do nosso modelo logístico, indicando que o modelo não conseguiu uma performance muito alta.

Essas medidas correspondem apenas ao do modelo polinomial, mas poderíamos calcular também para o linear. Entretanto, como queremos analisar diversos outros fatores além destes, podemos partir para um algoritmo mais geral, isto é, podemos executar o mesmo procedimento para cada índice econômico diário, diversos valores de retardo $k$.

Mas, antes disso, devemos ver quais índices iremos analisar:

In [17]:
economic_data.describe()

,ipca,igpm,inpc,selic_meta,international_reserve,pnad,cdi,gdp,dollar,employment,gov_debt,consumer_confidence,ibovespa
count,14.000000,14.000000,14.000000,440.000000,304.000000,13.000000,304.000000,14.000000,304.000000,7.000000,13.000000,14.000000,3.000000
mean,0.177857,0.724286,0.198571,4.709659,365819.200658,11.923077,0.017924,135246.571429,4.464194,179.415714,41.007692,113.320714,1.410000
std,0.366778,0.818993,0.367211,1.335170,16605.723326,0.679649,0.005106,21334.019762,0.608962,0.868655,1.486484,9.682220,2.415968
min,-0.380000,-0.670000,-0.250000,2.250000,338789.000000,11.000000,0.008442,107306.000000,3.740000,178.290000,37.950000,96.820000,-0.670000
25%,0.025000,0.285000,0.017500,3.750000,351742.500000,11.600000,0.014227,118631.250000,4.031825,178.825000,40.690000,108.265000,0.085000
50%,0.150000,0.580000,0.145000,4.500000,363855.500000,11.800000,0.017089,135412.500000,4.172350,179.250000,41.160000,112.080000,0.840000
75%,0.257500,1.130000,0.272500,6.000000,384912.750000,12.200000,0.022751,154674.250000,5.110750,180.030000,41.770000,120.605000,2.450000
max,1.150000,2.230000,1.220000,6.500000,390510.000000,13.300000,0.024620,158836.000000,5.937200,180.660000,43.630000,131.790000,4.060000


Como podemos observar, para este valor de $k$ e para o twitter de Jair bolsonaro, apenas alguns índices possuem uma quantidade útil de valores. Para outros casos isto irá continuar devido à continuidade dos dados. Portanto os índices de escolha são selic_meta, international_reserve, cdi e dollar. Com isso:

In [18]:
def getModelResults(kMax,economic_index,economic_data,tweets_data_cla,model_bootstrap=False,only_linear=False):
    # recebe os dados e parâmetros e executa todo o código apresentado até agora
    dates = economic_data.index[~np.isnan(economic_data[economic_index])]
    dates_index_tweets = [tweets_data_cla.index[row] in dates for row in range(0,len(tweets_data_cla))]
    model_data = tweets_data_cla.iloc[dates_index_tweets,:]
    model_data_values = model_data.values
    for date_ind in range(0,len(model_data.index)):
        row_values = model_data_values[date_ind,:]
        values_found = 1
        for k in range(1,kMax+1):
            cur_date = model_data.index[date_ind] - timedelta(days=k)
            if cur_date in tweets_data_cla.index:
                row_values = row_values + tweets_data_cla.values[list(tweets_data_cla.index).index(cur_date),:]
                values_found += 1
        model_data_values[date_ind,:] = row_values/values_found
    model_data = pd.DataFrame(model_data_values,columns=model_data.columns, index = model_data.index)
    model_data[economic_index] = economic_data[economic_index][~np.isnan(economic_data[economic_index])]
    eco_data = model_data[economic_index].values
    for row in range(len(eco_data)-1,0,-1):
        eco_data[row] = int(eco_data[row]>=eco_data[row-1])
    model_data[economic_index] = eco_data
    model_data = model_data.drop([model_data.index[0]])
    data_train, data_test = train_test_split(model_data,
                                             test_size = 0.2, 
                                             stratify=model_data[economic_index],random_state=42)
    x_train = data_train.drop(columns=economic_index)
    x_test = data_test.drop(columns=economic_index)
    y_train = np.array(data_train[economic_index])
    y_test = np.array(data_test[economic_index])        
    
    # caso queira gerar uma amostra bootstrap dos dados originais ao invés deles
    # isso será utilizado na próxima subseção
    if model_bootstrap:
        x_train, y_train = make_bootstrap_sample(x_train,y_train)
        y_train = y_train.ravel()
    
    x_train_t = x_train.as_matrix()
    y_train_t = np.array(y_train)
    x_test_t = x_test.as_matrix()
    y_test_t = np.array(y_test)
    lr = LogisticRegressionCV(solver='liblinear', multi_class='ovr',
                        penalty='l2', max_iter=100000, cv=10).fit(x_train_t,y_train_t)
    lr_train_accuracy = lr.score(x_train_t, y_train_t)
    lr_test_accuracy = lr.score(x_test_t, y_test_t)
    lr_class_accuracy = lr.score(np.concatenate([x_train_t[y_train_t==0,:],x_test_t[y_test_t==0,:]]),
                                 np.concatenate([y_train_t[y_train_t==0],y_test_t[y_test_t==0]]))
    scores = {'train': lr_train_accuracy,
              'test': lr_test_accuracy,
              'class_acu': lr_class_accuracy}
    
    # caso queiramos apenas o modelo linear, também será usado na próxima subseção
    if only_linear:
        return lr,np.array(x_train.columns)     
    
    columns_to_poly = ['retweet_count_mean', 'polarity_mean', 'subjectivity_mean','favorite_count_mean']
    columns_to_interact = ['media_type_photo', 'media_type_video','topic_1', 'topic_2', 'topic_3',
                           'topic_4', 'topic_5', 'topic_6','topic_7', 'topic_8', 'topic_9']
    poly_no_int = PolynomialFeatures(2, include_bias=False,interaction_only=False)
    poly_int = PolynomialFeatures(2, include_bias=False,interaction_only=True)
    dataframe_to_concat_train = []
    dataframe_to_concat_test = []
    for col in columns_to_poly:
        dataframe_to_concat_train.append(pd.DataFrame(poly_no_int.fit_transform(x_train[col].values.reshape(-1,1)),
                                               columns = [col,col+'**2']))
        dataframe_to_concat_test.append(pd.DataFrame(poly_no_int.fit_transform(x_test[col].values.reshape(-1,1)),
                                               columns = [col,col+'**2']))
    for col_poly in columns_to_poly:
        for col_int in columns_to_interact:
            k = poly_int.fit_transform(x_train[[col_poly,col_int]])
            dataframe_to_concat_train.append(pd.DataFrame(poly_int.fit_transform(x_train[[col_poly,col_int]])[:,2],
                                               columns = ['int_'+col_poly+'/'+col_int]))
            dataframe_to_concat_test.append(pd.DataFrame(poly_int.fit_transform(x_test[[col_poly,col_int]])[:,2],
                                               columns = ['int_'+col_poly+'/'+col_int])) 
    dataframe_to_concat_train.append(x_train[columns_to_interact].reset_index(drop=True))
    dataframe_to_concat_test.append(x_test[columns_to_interact].reset_index(drop=True))
    x_train_poly = pd.concat(dataframe_to_concat_train, axis=1)
    x_test_poly = pd.concat(dataframe_to_concat_test, axis=1)
    x_train_t_poly = x_train_poly.as_matrix()
    x_test_t_poly = x_test_poly.as_matrix()
    lrPoly = LogisticRegressionCV(solver='liblinear', multi_class='ovr',
                                penalty='l2', max_iter=100000, cv=10).fit(x_train_t_poly,y_train_t)
    lr_train_accuracy = lrPoly.score(x_train_t_poly, y_train_t)
    lr_test_accuracy = lrPoly.score(x_test_t_poly, y_test_t)
    lr_class_accuracy = lrPoly.score(np.concatenate([x_train_t_poly[y_train_t==0,:],x_test_t_poly[y_test_t==0,:]]),
                                 np.concatenate([y_train_t[y_train_t==0],y_test_t[y_test_t==0]]))
    scoresPoly = {'train': lr_train_accuracy,
              'test': lr_test_accuracy,
              'class_acu': lr_class_accuracy}
    
    # caso queiramos o polinomial
    if model_bootstrap and not only_linear:
        return lrPoly,np.array(x_train_poly.columns)
    
    # retorna uma lista com os resultados encontrados, sendo nessa lista os seguintes valores em ordem:
    #'eco_ind','k','per_y1','LogLin_acu_tr','LogLin_acu_te',
    #'LogLin_cla0_acu','LogPoly_acu_tr','LogPoly_acu_te','LogPoly_cla0_acu'
    return([economic_index,kMax, 
                   model_data[economic_index].sum()/len(model_data[economic_index]),
                   scores['train'],scores['test'],scores['class_acu'],
                   scoresPoly['train'],scoresPoly['test'],scoresPoly['class_acu']])

results = []
for economic_index in ['selic_meta','cdi','international_reserve','dollar']:
    for kMax in range(0,5):
        results.append(getModelResults(kMax,economic_index,economic_data,tweets_data_cla))
results = pd.DataFrame(results,columns=['eco_ind','k','per_y1','LogLin_acu_tr','LogLin_acu_te',
                                       'LogLin_cla0_acu','LogPoly_acu_tr','LogPoly_acu_te',
                                       'LogPoly_cla0_acu'])
results

,eco_ind,k,per_y1,LogLin_acu_tr,LogLin_acu_te,LogLin_cla0_acu,LogPoly_acu_tr,LogPoly_acu_te,LogPoly_cla0_acu
0,selic_meta,0,0.981735,0.982857,0.977273,0.000000,0.982857,0.977273,0.000000
1,selic_meta,1,0.981735,0.982857,0.977273,0.000000,0.982857,0.977273,0.000000
2,selic_meta,2,0.981735,0.982857,0.977273,0.000000,0.982857,0.977273,0.000000
3,selic_meta,3,0.981735,0.982857,0.977273,0.000000,0.982857,0.977273,0.000000
4,selic_meta,4,0.981735,0.982857,0.977273,0.000000,0.982857,0.977273,0.000000
5,cdi,0,0.973510,0.975104,0.967213,0.000000,0.975104,0.967213,0.000000
6,cdi,1,0.973510,0.975104,0.967213,0.000000,0.975104,0.967213,0.000000
7,cdi,2,0.973510,0.975104,0.967213,0.000000,0.975104,0.967213,0.000000
8,cdi,3,0.973510,0.975104,0.967213,0.000000,0.975104,0.967213,0.000000
9,cdi,4,0.973510,0.975104,0.967213,0.000000,0.975104,0.967213,0.000000


**Como realizar a leitura do dataframe acima:**
Cada linha corresponde a um valor de $k$ diferente ou um índice econômico diferente. Nossas colunas representam:
1. eco_ind é o nosso índice econômico;
2. $k$ é o nosso valor de retardo;
3. per_y1 é a porcentagem de valores $1$ no nosso y, isto é, quantas vezes o índice e econômico cresceu com relação ao total;
4. LogLin_acu_tr é a acurácia do modelo Logístico Linear no treino;
5. LogLin_acu_te é a acurácia do modelo Logístico Linear no teste;
6. LogLin_cla0_acu é a acurácia do modelo Logístico Linear na classe $0$ em ambos treino e teste;
7. LogPoly_acu_tr é a acurácia do modelo Logístico Polinomial no treino;
8. LogPoly_acu_te é a acurácia do modelo Logístico Polinomial no teste;
9. LogPoly_cla0_acu é a acurácia do modelo Logístico Polinomial na classe $0$ em ambos treino e teste;

O código acima é apenas tudo que fizermos anteriormente concatenados de forma à executarmos a mesma análise nos outros índices e valores de $k$. Uma análise a primeira instância é de que:

1. Os índices econômicos selic_meta e cdi possuem 97% de valores $1$, de modo que nosso modelo falha pois classifica qualquer entrada em um aumento (valor $1$). Isso pode ser visto através da coluna de acurácia na classe $0$;
2. Para a reserva internacional, $k=2$ nos forneceu a maior acurácia no teste em ambos modelos (linear e polinomial) com $0.606557$ e $0.639344$, respectivamente.
3. Para o câmbio do dólar, $k=1$ nos forneceu a maior acurácia no teste no modelo $1$, com $0.573770$ enquanto $k=4$ no modelo $2$ nos deu $0.622951$.

## Significância dos coeficientes

A partir dos três parâmetros juntamente com os índices que mostraram resultados interessantes anteriormente, esses são

1. Para a reserva internacional $k=2$;
2. Para o câmbio do dólar, $k=1$ para o modelo linear;
3. Para o câmbio do dólar, $k=4$ no modelo polinomial;

podemos analisar quais são os coeficientes que estão sendo mais significantes nos nossos modelos. O método funciona da seguinte maneira:

- Primeiro geramos várias bootstrap samples e fitamos o modelo em cada uma delas;
- Nisso guardamos os valores dos coeficientes em cada um dos bootstraps;
- Considerando que os coeficientes vêm de uma distribuição normal, podemos verificar se o intervalo de $95%$ dessa distribuição contém o valor $0$. Se sim, esse coeficiente não é significante.

Portanto:

In [21]:
## função para gerar uma amostra bootstrap aleatória
def make_bootstrap_sample(dataset_X, dataset_y, size = None):
    # dataset_x é o dataframe de X e dataset_y é um np.array(:,)
    # o valor de retorno é uma amostra aleatória dos dados de tamanho igual à entrada
    if not size: size = len(dataset_X)
    if len(dataset_X) != len(dataset_y):
        raise Exception("Data size must match between dataset_X and dataset_y")
    ind = np.random.randint(0,size,size)
    bootstrap_dataset_X = dataset_X.iloc[ind,:]
    bootstrap_dataset_y = np.array(dataset_y[ind])
    bootstrap_dataset_y = bootstrap_dataset_y.reshape(-1,1)
    return (bootstrap_dataset_X, bootstrap_dataset_y)

def calculate_coefficients(columns_names, model):
    # retorna o dicionário com a chave sendo o nome do preditor e
    # o valor o coeficiente referente
    values = np.array(model.coef_)
    values = values.reshape(-1,1)
    names = list(columns_names)
    coefficients_dictionary = {names[i]:float(values[i]) for i in range(0,len(names))}
    coefficients_dictionary['intercept'] = float(model.intercept_)
    return coefficients_dictionary


def get_significant_predictors(regression_coefficients, significance_level):
    # regression coefficients é a lista de dicionários gerados pelo calculate_coefficients
    # signifance_level é o grau de significância
    # o retorno da função são os coeficientes que passam no teste de significância
    sdCoef = norm.ppf(1-significance_level/2)
    features = list(regression_coefficients[0].keys())
    significant_coefficients = []
    for feat in features:
        values = [dic[feat] for dic in regression_coefficients]
        mean = np.mean(values)
        sdev = np.std(values)
        if mean-sdCoef*sdev>0 or mean+sdCoef*sdev<0:
            significant_coefficients.append(feat)
    return significant_coefficients

# valores para testar: k / economic_index / é o modelo linear?
models = {1:[2,'international_reserve',True],
           2:[2,'international_reserve',False],
           3:[1,'dollar',True],
           4:[4,'dollar',False]}

models_sig_coe = {}

# para cada modelo
for i in range(1,len(models)+1):
    k, economic_index, is_linear = models[i]
    regression_coefficients = []
    # rode n_samples=100 bootstraps e armezene os dicionários
    n_samples = 100
    for j in range(0,n_samples):
        # uma fórmula para sabermos quanto falta
        if(int((i-1)*n_samples+j)%(n_samples*len(models)/10)==0): print(str(round(int(((i-1)*n_samples+j)/(len(models))),2))+'%') 
        # pega o nosso modelo e o nome dos coeficientes
        model,columns_names = getModelResults(k,economic_index,economic_data,
                                              tweets_data_cla,model_bootstrap=True,only_linear=is_linear)
        coefficients_dictionary = calculate_coefficients(columns_names, model)
        # armazena na lista o dicionário
        regression_coefficients.append(coefficients_dictionary)
    # pegue os coeficientes significativos através dessa lista de dicionários
    significance_level = 0.25
    significant_coefficients = get_significant_predictors(regression_coefficients, significance_level)
    models_sig_coe[i] = significant_coefficients

0%
10%
20%
30%
40%
50%
60%
70%
80%
90%


In [22]:
models_sig_coe

{1: ['polarity_mean'],
 2: ['subjectivity_mean**2',
  'int_retweet_count_mean/topic_4',
  'int_polarity_mean/media_type_photo',
  'int_polarity_mean/topic_5',
  'int_polarity_mean/topic_8',
  'int_subjectivity_mean/media_type_photo',
  'int_subjectivity_mean/topic_2',
  'topic_4'],
 3: ['polarity_mean'],
 4: ['polarity_mean**2',
  'subjectivity_mean**2',
  'int_polarity_mean/topic_5',
  'int_subjectivity_mean/topic_5',
  'media_type_photo',
  'media_type_video',
  'topic_5',
  'intercept']}

Obtivemos os seguintes resultados:

- Para os dois modelos lineares (1 e 3), a polaridade média é o único preditor com significância alfa;
- Para o modelo polinomial, os tópicos mais influentes são o 2, 4, 5 e 8 na reserva internacional, juntamente com as dummies de mídia.
- Para o modelo polinomial do câmbio do dólar, é interessante ver que ambas polaridade e subjetividade quadrática afetam o modelo polinomial, também interessante verificar que o tópico 0 (por causa do intercept) e 5 também são significantes.